### Vanilla CTR Zeroshot

In [2]:
import pickle
import json
import pandas as pd
import numpy as np
import os
import time
from dotenv import load_dotenv
import re

In [3]:
inference_path = 'zeroshot_inference_test.json'
with open(inference_path, 'r') as f:
    inference_dict = json.load(f)
print(len(inference_dict))

1240


In [4]:
ratings_path = '../fashion_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1452816000
1,1,0,4.0,1453766400
2,2,0,3.0,1453766400
3,3,0,4.0,1453766400
4,4,0,4.0,1453766400


In [5]:
pattern = r"Prediction:(?:\s+)?(Yes|No)\b"
invalid_users = []
cnt = cntInvalid = cntY = cntN = 0
user_inference_dict = dict()
for user, inference in inference_dict.items():
    cnt += 1
    # print(user, inference)
    # Use re.findall to find all matches in the string
    matches = re.findall(pattern, inference, re.IGNORECASE)
    if len(matches) == 0:
        cntInvalid += 1
        invalid_users.append(user)
        user_inference_dict[user] = 1
    for match in matches:
        if match == 'Yes':
            cntY += 1
            user_inference_dict[user] = 1
            break
        elif match == 'No':
            cntN += 1
            user_inference_dict[user] = 0
            break
    # print('*'*100)
    # if cnt == 10:
    #     break
print(f"Yes: {cntY} -- No: {cntN} -- Invalid: {cntInvalid}")
print("invalid_users:", invalid_users)
print(len(user_inference_dict))

Yes: 772 -- No: 299 -- Invalid: 169
invalid_users: ['6', '32', '34', '39', '45', '46', '48', '52', '59', '60', '67', '75', '81', '90', '102', '115', '116', '124', '132', '143', '145', '148', '163', '172', '173', '176', '189', '193', '194', '220', '238', '240', '241', '244', '252', '274', '277', '280', '286', '289', '298', '302', '313', '314', '319', '326', '330', '344', '346', '350', '353', '356', '364', '380', '382', '387', '394', '398', '409', '414', '420', '428', '432', '443', '446', '448', '462', '474', '475', '477', '483', '494', '523', '535', '545', '555', '569', '570', '571', '577', '582', '601', '619', '636', '638', '655', '659', '662', '665', '673', '700', '701', '705', '713', '718', '734', '741', '756', '764', '766', '773', '798', '799', '803', '811', '813', '820', '829', '830', '848', '852', '860', '869', '870', '873', '875', '876', '878', '879', '895', '898', '903', '908', '911', '912', '914', '917', '928', '929', '935', '939', '940', '941', '954', '959', '963', '967', '991

In [6]:
user_test_movie_rating_df = ratings_df.groupby('user').nth(-1)
user_test_movie_rating_df.head()

,item,user,rating,timestamp
7,7,0,3.0,1500854400
17,17,1,5.0,1470096000
25,25,2,4.0,1462924800
33,25,3,4.0,1495065600
41,25,4,5.0,1509580800


In [7]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    original_rating = user_test_movie_rating_df[user_test_movie_rating_df['user'] == int(user)]['rating'].values[0]
    # print(original_rating)
    if original_rating > 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[132 157]
 [167 784]]
AUC: 0.6405713890677815
Precision: 0.8331562167906482
Recall: 0.8243953732912723
Accuracy: 0.7387096774193549


In [5]:
user_valid_movie_rating_df = ratings_df.groupby('user').nth(-2)
user_valid_movie_rating_df.head()

,item,user,rating,timestamp
6,5,0,5.0,1404604800
13,11,1,5.0,1504656000
20,11,2,4.0,1506643200
26,19,3,5.0,1493337600
32,19,4,5.0,1524614400


In [6]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    original_rating = user_valid_movie_rating_df[user_valid_movie_rating_df['user'] == int(user)]['rating'].values[0]
    # print(original_rating)
    if original_rating > 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[ 29  15]
 [118 454]]
AUC: 0.7263986013986015
Precision: 0.9680170575692963
Recall: 0.7937062937062938
Accuracy: 0.7840909090909091
